In [3]:
import numpy as np
import os

In [133]:
'''
Implements the WEAT tests 
Adapted from https://github.com/W4ngatang/sent-bias/blob/master/sentbias/weat.py
'''
import logging as log
import math
import itertools as it
import numpy as np
import scipy.special
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity
import argparse

# X and Y are two sets of target words of equal size.
# A and B are two sets of attribute words.


class Test:
    def __init__(self, X, Y, A, B, names=None):
        """
        A WEAT Test.
        :param X: A set of target embeddings
        :param Y: A set of target embeddings
        :param A: A set of attribute embeddings
        :param B: A set of attribute embeddings
        :param names: Optional set of names for X, Y, A, and B, in order
        :return: the effect size and p-value
        """
        self.X = X
        self.Y = Y
        self.A = A
        self.B = B
        self.names = names if names is not None else ["X", "Y", "A", "B"]
        self.reset_calc()

    def reset_calc(self):
        log.info("Computing cosine similarities...")
        self.similarity_matrix = self.similarities()
        self.s_AB = None
        self.calc_s_AB()

    def run(self, randomized=False, **kwargs):
        """
        Run the test.
        """
        if randomized:
            X_orig = self.X
            Y_orig = self.Y
            A_orig = self.A
            B_orig = self.B
            D = np.concatenate((self.X, self.Y, self.A, self.B))
            np.random.shuffle(D)
            self.X = D[:X_orig.shape[0],:]
            self.Y = D[X_orig.shape[0]:2*X_orig.shape[0],:]
            self.A = D[2*X_orig.shape[0]:2*X_orig.shape[0]+A_orig.shape[0], :]
            self.B = D[2*X_orig.shape[0]+A_orig.shape[0]:, :]
            self.reset_calc()


        p = self.p(**kwargs)
        log.info("pval: %g", p)

        log.info("computing effect size...")
        e = self.effect_size()
        log.info("esize: %g", e)

        if randomized:
            self.X = X_orig
            self.Y = Y_orig
            self.A = A_orig
            self.B = B_orig
            self.reset_calc()
        return e, p  

    def similarities(self):
        """
        :return: an array of size (len(XY), len(AB)) containing cosine similarities
        between items in XY and items in AB.
        """
        XY = np.concatenate((self.X, self.Y))
        AB = np.concatenate((self.A, self.B))
        return cosine_similarity(XY, AB)

    def calc_s_AB(self):
        self.s_AB = self.s_wAB(np.arange(self.similarity_matrix.shape[0]))

    def s_wAB(self, w):
        """
        Return vector of s(w, A, B) across w, where
            s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
        :param w: Mask on the XY axis of similarity matrix
        """
        return self.similarity_matrix[w, :self.A.shape[0]].mean(axis=1) - self.similarity_matrix[w, self.A.shape[0]:].mean(axis=1)

    def s_XAB(self, mask):
        r"""
        Given indices of target concept X and precomputed s_wAB values,
        return slightly more computationally efficient version of WEAT
        statistic for p-value computation.
        Caliskan defines the WEAT statistic s(X, Y, A, B) as
            sum_{x in X} s(x, A, B) - sum_{y in Y} s(y, A, B)
        where s(w, A, B) is defined as
            mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
        The p-value is computed using a permutation test on (X, Y) over all
        partitions (X', Y') of X union Y with |X'| = |Y'|.
        However, for all partitions (X', Y') of X union Y,
            s(X', Y', A, B)
          = sum_{x in X'} s(x, A, B) + sum_{y in Y'} s(y, A, B)
          = C,
        a constant.  Thus
            sum_{x in X'} s(x, A, B) + sum_{y in Y'} s(y, A, B)
          = sum_{x in X'} s(x, A, B) + (C - sum_{x in X'} s(x, A, B))
          = C + 2 sum_{x in X'} s(x, A, B).
        By monotonicity,
            s(X', Y', A, B) > s(X, Y, A, B)
        if and only if
            [s(X', Y', A, B) - C] / 2 > [s(X, Y, A, B) - C] / 2,
        that is,
            sum_{x in X'} s(x, A, B) > sum_{x in X} s(x, A, B).
        Thus we only need use the first component of s(X, Y, A, B) as our
        test statistic.
        :param mask: some random X partition of XY - in the form of a mask on XY
        """
        return self.s_AB[mask].sum()

    def s_XYAB(self, X, Y):
        r"""
        Given indices of target concept X and precomputed s_wAB values,
        the WEAT test statistic for p-value computation.
        :param X: Mask for XY indicating the values in partition X
        :param Y: Mask for XY indicating the values in partition Y
        """
        return self.s_XAB(X) - self.s_XAB(Y)

    def p(self, n_samples=10000, parametric=False):
        """ 
        Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
        """
        assert self.X.shape[0] == self.Y.shape[0]
        size = self.X.shape[0]

        XY = np.concatenate((self.X, self.Y))

        if parametric:
            log.info('Using parametric test')
            s = self.s_XYAB(np.arange(self.X.shape[0]), np.arange(self.X.shape[0], self.X.shape[0]+self.Y.shape[0]))

            log.info('Drawing {} samples'.format(n_samples))
            samples = []
            for _ in range(n_samples):
                a = np.arange(XY.shape[0])
                np.random.shuffle(a)
                Xi = a[:size]
                Yi = a[size:]
                assert len(Xi) == len(Yi)
                si = self.s_XYAB(Xi, Yi)
                samples.append(si)

            # Compute sample standard deviation and compute p-value by
            # assuming normality of null distribution
            log.info('Inferring p-value based on normal distribution')
            (shapiro_test_stat, shapiro_p_val) = scipy.stats.shapiro(samples)
            log.info('Shapiro-Wilk normality test statistic: {:.2g}, p-value: {:.2g}'.format(
                shapiro_test_stat, shapiro_p_val))
            sample_mean = np.mean(samples)
            sample_std = np.std(samples, ddof=1)
            log.info('Sample mean: {:.2g}, sample standard deviation: {:.2g}'.format(
                sample_mean, sample_std))
            p_val = scipy.stats.norm.sf(s, loc=sample_mean, scale=sample_std)
            return p_val

        else:
            log.info('Using non-parametric test')
            s = self.s_XAB(np.arange(self.X.shape[0]))
            total_true = 0
            total_equal = 0
            total = 0

            num_partitions = int(scipy.special.binom(2 * self.X.shape[0], self.X.shape[0]))
            if num_partitions > n_samples:
                # We only have as much precision as the number of samples drawn;
                # bias the p-value (hallucinate a positive observation) to
                # reflect that.
                total_true += 1
                total += 1
                log.info('Drawing {} samples (and biasing by 1)'.format(n_samples - total))
                for i in range(n_samples - 1):
                    a = np.arange(XY.shape[0])
                    np.random.shuffle(a)
                    Xi = a[:size]
                    assert 2 * len(Xi) == len(XY)
                    si = self.s_XAB(Xi)
                    if si > s:
                        total_true += 1
                    elif si == s:  # use conservative test
                        total_true += 1
                        total_equal += 1
                    total += 1
            else:
                log.info('Using exact test ({} partitions)'.format(num_partitions))
                # iterate through all possible X-length combinations of the indices of XY
                for Xi in it.combinations(np.arange(XY.shape[0]), self.X.shape[0]):
                    assert 2 * len(Xi) == len(XY)
                    si = self.s_XAB(np.array(Xi))
                    if si > s:
                        total_true += 1
                    elif si == s:  # use conservative test
                        total_true += 1
                        total_equal += 1
                    total += 1

            if total_equal:
                log.warning('Equalities contributed {}/{} to p-value'.format(total_equal, total))
            return total_true / total

    def effect_size(self):
        """
        Compute the effect size, which is defined as
            [mean_{x in X} s(x, A, B) - mean_{y in Y} s(y, A, B)] /
                [ stddev_{w in X u Y} s(w, A, B) ]
        args:
            - X, Y, A, B : sets of target (X, Y) and attribute (A, B) indices
        """
        numerator = np.mean(self.s_wAB(np.arange(self.X.shape[0]))) - np.mean(self.s_wAB(np.arange(self.X.shape[0], self.similarity_matrix.shape[0])))
        denominator = np.std(self.s_AB, ddof=1)
        return numerator / denominator





In [134]:
def load_features(folder, pca):
    if pca==0.0:
        pca_path = 'no_pca/'
    else:
        pca_path = 'pca/'

    features = dict()
    features_pt = os.listdir(folder+'/pretrained_features/'+pca_path)
    features_ft = os.listdir(folder+'/finetuned_features/'+pca_path)
    for file_name in features_pt:
        features[os.path.splitext(file_name)[0]] = np.load(folder + '/pretrained_features/' + pca_path + file_name, allow_pickle=True)
    for file_name in features_ft:
        features[os.path.splitext(file_name)[0]] = np.load(folder + '/finetuned_features/' + pca_path + file_name, allow_pickle=True)
    return features


def load_ieat_features(model_name, dataset):
    trials = os.listdir('../experiments/'+dataset+'/'+model_name) 
    if 'orig' in trials:
        trials.remove('orig')
    if '.ipynb_checkpoints' in trials:
        trials.remove('.ipynb_checkpoints')
    all_features = []
    for trial in trials:
        features = load_features('../experiments/'+dataset+'/'+model_name+'/'+trial, pca=0.0)
        if bool(features):
            all_features.append(features)
    return all_features


def run_test(X, Y, A, B, model_name, dataset, print_stats=False, n_samples=10000):
    features = load_ieat_features(model_name, dataset)
    np.random.seed(38)
    pvals = []
    effect_sizes = []
    for feature_trial in features:
        cap = min(feature_trial[X].shape[0], feature_trial[Y].shape[0])
        test = Test(feature_trial[X][:cap], feature_trial[Y][:cap], feature_trial[A], feature_trial[B])
        pval = test.run(n_samples=n_samples)
        effect_sizes.append(pval[0])
        pvals.append(pval[1])
    pval_mean = np.mean(pvals)
    effect_size_mean = np.mean(effect_sizes)
    if print_stats == True:
        print("Model Name: ", model_name)
        print("Analysis Set: ", dataset)
        print("Effect Size Mean: ", effect_size_mean)
        print("Pval Mean: ", pval_mean)
    return pval_mean, effect_size_mean



In [135]:
import pandas as pd

def coco_ieat(model_names):
    counter = 0
    for model_name in model_names:
        print(model_name)
        # refrig vs surfb
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'ra_pt', 'sa_pt', model_name, 'coco')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'ra_ft', 'sa_ft', model_name, 'coco')
        if counter == 0:
            refrig_surfb= {'X':['Man'], 'Y':['Woman'], 'A':['Refrig'], 'B':['Surfb'],
                          'Model': [model_name], 'iEAT d PT': [effect_size_pt], 'iEAT p PT': [pval_pt], 
                          'iEAT d FT': [effect_size_ft], 'iEAT p FT': [pval_ft]}
            df = pd.DataFrame(data=refrig_surfb)
        else:
            refrig_surfb= {'X':'Man', 'Y':'Woman', 'A':'Refrig', 'B':'Surfb',
                          'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                          'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
            df = df.append(refrig_surfb, ignore_index=True)

        # refrig vs car
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'ra_pt', 'ca_pt', model_name, 'coco')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'ra_ft', 'ca_ft', model_name, 'coco')

        refrig_car = {'X':'Man', 'Y':'Woman', 'A':'Refrig', 'B':'Car',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}

        df = df.append(refrig_car, ignore_index=True)

        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'sa_pt', 'ca_pt', model_name, 'coco')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'sa_ft', 'ca_ft', model_name, 'coco')

        surfb_car = {'X':'Man', 'Y':'Woman', 'A':'Surfb', 'B':'Car',
                      'Model': model_name, 'iEAT d PT':effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
        df = df.append(surfb_car, ignore_index=True)
        
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'sa_pt', 'ca_pt', model_name, 'coco')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'sa_ft', 'ca_ft', model_name, 'coco')

        counter +=1
    
    display(df)
    
    
    return df
    


In [136]:
coco = coco_ieat(['clip', 'bit_resnet50', 'resnet18', 'resnet50', 'simclr_resnet50','moco_resnet50', 'virtex_resnet50'])
print(coco.to_latex(multirow=True, bold_rows=True))

clip
bit_resnet50
resnet18


resnet50
simclr_resnet50
moco_resnet50
virtex_resnet50


,X,Y,A,B,Model,iEAT d PT,iEAT p PT,iEAT d FT,iEAT p FT
0,Man,Woman,Refrig,Surfb,clip,-0.210225,0.688380,-0.383904,0.773140
1,Man,Woman,Refrig,Car,clip,-0.327352,0.780660,-0.402179,0.816600
2,Man,Woman,Surfb,Car,clip,-0.356431,0.802480,-0.177943,0.628400
3,Man,Woman,Refrig,Surfb,bit_resnet50,0.645048,0.058167,0.009005,0.493000
4,Man,Woman,Refrig,Car,bit_resnet50,0.411640,0.163433,0.181887,0.332067
5,Man,Woman,Surfb,Car,bit_resnet50,-0.564415,0.912800,0.267277,0.262400
6,Man,Woman,Refrig,Surfb,resnet18,0.131729,0.375400,0.269507,0.268267
7,Man,Woman,Refrig,Car,resnet18,0.094906,0.406800,0.070855,0.438200
8,Man,Woman,Surfb,Car,resnet18,-0.081718,0.577200,-0.209579,0.655900
9,Man,Woman,Refrig,Surfb,resnet50,0.378689,0.183475,0.089305,0.416225


\begin{tabular}{llllllrrrr}
\toprule
{} &    X &      Y &       A &      B &            Model &  iEAT d PT &  iEAT p PT &  iEAT d FT &  iEAT p FT \\
\midrule
\textbf{0 } &  Man &  Woman &  Refrig &  Surfb &             clip &  -0.210225 &   0.688380 &  -0.383904 &   0.773140 \\
\textbf{1 } &  Man &  Woman &  Refrig &    Car &             clip &  -0.327352 &   0.780660 &  -0.402179 &   0.816600 \\
\textbf{2 } &  Man &  Woman &   Surfb &    Car &             clip &  -0.356431 &   0.802480 &  -0.177943 &   0.628400 \\
\textbf{3 } &  Man &  Woman &  Refrig &  Surfb &     bit\_resnet50 &   0.645048 &   0.058167 &   0.009005 &   0.493000 \\
\textbf{4 } &  Man &  Woman &  Refrig &    Car &     bit\_resnet50 &   0.411640 &   0.163433 &   0.181887 &   0.332067 \\
\textbf{5 } &  Man &  Woman &   Surfb &    Car &     bit\_resnet50 &  -0.564415 &   0.912800 &   0.267277 &   0.262400 \\
\textbf{6 } &  Man &  Woman &  Refrig &  Surfb &         resnet18 &   0.131729 &   0.375400 &   0.269507 &   0.26

In [138]:
# surfb vs car
# surfb vs fashion
# surfb vs mammal
# car vs fashion
# car vs. mammal
# mammal vs. fashion


def openimages_ieat(model_names):
    counter = 0
    for model_name in model_names:
        print(model_name)
        # surfb vs car
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'sa_pt', 'ca_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'sa_ft', 'ca_ft', model_name, 'openimages')
        if counter == 0:
            surfb_car= {'X':['Man'], 'Y':['Woman'], 'A':['Surfb'], 'B':['Car'],
                          'Model': [model_name], 'iEAT d PT': [effect_size_pt], 'iEAT p PT': [pval_pt], 
                          'iEAT d FT': [effect_size_ft], 'iEAT p FT': [pval_ft]}
            df = pd.DataFrame(data=surfb_car)
        else:
            surfb_car= {'X':'Man', 'Y':'Woman', 'A':'Surfb', 'B':'Car',
                          'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                          'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
            df = df.append(surfb_car, ignore_index=True)

        # surfb vs fashion
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'sa_pt', 'fa_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'sa_ft', 'fa_ft', model_name, 'openimages')

        surfb_fashion = {'X':'Man', 'Y':'Woman', 'A':'Surfb', 'B':'Fashion',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}

        df = df.append(surfb_fashion, ignore_index=True)
        
        # surfb vs mammal

        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'sa_pt', 'ma_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'sa_ft', 'ma_ft', model_name, 'openimages')

        surfb_mammal = {'X':'Man', 'Y':'Woman', 'A':'Surfb', 'B':'Mammal',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
        df = df.append(surfb_mammal, ignore_index=True)
        
        # car vs fashion
        
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'ca_pt', 'fa_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'ca_ft', 'fa_ft', model_name, 'openimages')

        car_fashion = {'X':'Man', 'Y':'Woman', 'A':'Car', 'B':'Fashion',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
        df = df.append(car_fashion, ignore_index=True)
        
        # car vs. mammal
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'ca_pt', 'ma_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'ca_ft', 'ma_ft', model_name, 'openimages')

        car_mammal = {'X':'Man', 'Y':'Woman', 'A':'Car', 'B':'Mammal',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
        df = df.append(car_mammal, ignore_index=True)
        
        counter +=1
        
        # mammal vs. fashion
        
        pval_pt, effect_size_pt = run_test('man_pt', 'woman_pt', 'ma_pt', 'fa_pt', model_name, 'openimages')
        pval_ft, effect_size_ft = run_test('man_ft', 'woman_ft', 'ma_ft', 'fa_ft', model_name, 'openimages')

        mammal_fashion = {'X':'Man', 'Y':'Woman', 'A':'Mammal', 'B':'Fashion',
                      'Model': model_name, 'iEAT d PT': effect_size_pt, 'iEAT p PT': pval_pt, 
                      'iEAT d FT': effect_size_ft, 'iEAT p FT': pval_ft}
        df = df.append(mammal_fashion, ignore_index=True)
        
        counter +=1
    
    display(df)
    
    
    return df

In [139]:
openimages = openimages_ieat(['bit_resnet50', 'clip', 'resnet18', 'resnet50'])

bit_resnet50
clip
resnet18


resnet50


,X,Y,A,B,Model,iEAT d PT,iEAT p PT,iEAT d FT,iEAT p FT
0,Man,Woman,Surfb,Car,bit_resnet50,0.346063,0.0010,-0.060876,0.7018
1,Man,Woman,Surfb,Fashion,bit_resnet50,0.740230,0.0001,1.039939,0.0001
2,Man,Woman,Surfb,Mammal,bit_resnet50,0.519440,0.0001,0.809373,0.0001
3,Man,Woman,Car,Fashion,bit_resnet50,0.518999,0.0001,0.996554,0.0001
4,Man,Woman,Car,Mammal,bit_resnet50,0.091239,0.2192,0.787732,0.0001
5,Man,Woman,Mammal,Fashion,bit_resnet50,0.605058,0.0001,0.508390,0.0001
6,Man,Woman,Surfb,Car,clip,0.271835,0.0100,-0.447669,0.9999
7,Man,Woman,Surfb,Fashion,clip,1.031089,0.0001,0.583105,0.0001
8,Man,Woman,Surfb,Mammal,clip,0.557674,0.0001,0.551149,0.0001
9,Man,Woman,Car,Fashion,clip,0.961912,0.0001,0.763488,0.0001
